In [11]:
from langchain.llms import OpenAI
import os
from dotenv import load_dotenv
from pdfminer.high_level import extract_text
import re
from pdfminer.high_level import extract_text
from input import job_title,job_description,Company_Name
env_path = 'Config.env'
load_dotenv(dotenv_path=env_path)

Resume_path= "C:/Users/hrith.DESKTOP-75K32P0/OneDrive/Documents/Hrithik_Files_Desktop/FT_Resumes_NEU/Hrithik_Resume_Data_Engineer.pdf"

# Set up OpenAI LLM
openai_api_key = os.getenv('OPENAI_API_KEY')  # Store your API key in environment variable
gemini_api_key=os.getenv('GEMINI_API_KEY')


In [20]:

def extract_resume_text(pdf_path):
    return extract_text(pdf_path)

resume_text = extract_resume_text(Resume_path)


def remove_date_ranges(text):
    # Regex pattern for date ranges (e.g., "Sep 2021 - Dec 2021" or "Sep 2021 to Dec 2021")
    date_pattern = r"\b(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{4}\s*(?:-|to)\s*(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\s+\d{4}"
    
    # Remove date ranges
    return re.sub(date_pattern, "", text)

def extract_resume_sections(pdf_path):
    # Extract text from PDF
    full_text = extract_text(pdf_path)

    # Remove date ranges from the text
    full_text = remove_date_ranges(full_text)

    # Use regex patterns to identify sections
    education_pattern = r"(Education\s*[\s\S]+?)(?=(Skills|Work Experience|Projects|$))"
    skills_pattern = r"(Skills\s*[\s\S]+?)(?=(Education|Work Experience|Projects|$))"
    work_experience_pattern = r"(Work Experience\s*[\s\S]+?)(?=(Education|Skills|Projects|$))"
    projects_pattern = r"(Projects\s*[\s\S]+?)(?=(Education|Skills|Work Experience|$))"

    # Find matches using regex
    education = re.search(education_pattern, full_text, re.IGNORECASE)
    skills = re.search(skills_pattern, full_text, re.IGNORECASE)
    work_experience = re.search(work_experience_pattern, full_text, re.IGNORECASE)
    projects = re.search(projects_pattern, full_text, re.IGNORECASE)

    # Extract text if matches are found
    education_text = education.group(1).strip() if education else "Education section not found"
    skills_text = skills.group(1).strip() if skills else "Skills section not found"
    work_experience_text = work_experience.group(1).strip() if work_experience else "Work Experience section not found"
    projects_text = projects.group(1).strip() if projects else "Projects section not found"

    return {
        "Education": education_text,
        "Skills": skills_text,
        "Work Experience": work_experience_text,
        "Projects": projects_text
    }

resume_sections = extract_resume_sections(Resume_path)

def create_email_prompt(resume_sections,job_title,company_name,job_description):
    education = resume_sections["Education"]
    work_experience = resume_sections["Work Experience"]
    projects = resume_sections["Projects"]
    skills = resume_sections["Skills"]

    prompt = (
        f"You are a professional in email writing and technically strong when it comes to writing emails on behalf "
        f"of aspiring Data Engineers, Data Scientists, Data Analysts, and Machine Learning Engineers to Hiring Companies. "
        f"Write a professional and appealing email for Hiring Managers and Technical Recruiters for my job application. "
        f"The job title I am applying for is {job_title} at {company_name}.\n\n"
        f"The job description is as follows:\n{job_description}\n\n"
        f"My Professional Profile / Resume is divided into 4 sections:\n\n"
        f"My Education is as follows: {resume_sections['Education']}.\n\n"
        f"The Work Experience is as follows: {resume_sections['Work Experience']}.\n\n"
        f"My Personal and academic Projects are as follows: {resume_sections['Projects']}.\n\n"
        f"My Skills are as follows: {resume_sections['Skills']}.\n\n"
        f"The email should include a subject line and a body. The tone should be polite and professional, "
        f"and the email should request consideration as an eligible candidate, highlighting how my skills and experience align "
        f"with the job requirements."
    )

    return prompt


def write_response_to_file(response, file_path):
    with open(file_path, 'w') as file:
        file.write(response)


In [62]:
openai_api_key = os.getenv('OPENAI_API_KEY')  # Store your API key in environment variable
llm = OpenAI(model_name="gpt-3.5-turbo", api_key=openai_api_key)
def generate_email(resume_sections):
    prompt = create_profile_email_prompt(resume_sections)
    email_response = llm(prompt)
    #print(email_response)
    return email_response

In [18]:
import google.generativeai as genai
import os

genai.configure(api_key=gemini_api_key)

model = genai.GenerativeModel('gemini-1.5-flash')
prompt = create_email_prompt(resume_sections,job_title,Company_Name,job_description)
response = model.generate_content(prompt)

In [21]:
write_response_to_file(response.text, 'Email_Response.txt')

In [19]:
response.text

"## Subject: Data Engineer Application - [Your Name] - Eager to Contribute at Relanto\n\nDear [Hiring Manager Name],\n\nI am writing to express my keen interest in the Data Engineer position at Relanto, as advertised on [platform where you found the job posting]. With my proven experience in designing, developing, and optimizing data systems and pipelines, coupled with my strong foundation in SQL, data modeling, and cloud-based data platforms, I believe I possess the skills and expertise to make a significant contribution to your team.\n\nMy background in Data Engineering, as demonstrated in my resume, showcases a diverse range of accomplishments. At [Previous Company], I implemented an ETL pipeline to replicate data from SQL Server to Snowflake, enabling the BI team to build an audience sizing tool for marketing campaigns. I also developed an optimized geo-location matching algorithm, reducing processing time by 70%. \n\nFurthermore, my academic projects, such as the Early Sepsis Pred

In [9]:
response2= model.generate_content('Rephrase the previous response with my name as hrithik Sarda, Job title as Data Engineer and Company Name as Bright Horizons and Hiring Manager as Mr. Puru Ethirajan')

In [10]:
response2.text

'Please provide me with the previous response so I can rephrase it with the information you provided. I need the original text to accurately incorporate your name, job title, company, and hiring manager. \n'